In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scripts.preprocess import preprocess_data, encode_sequences, read_data
from models.LSTM import create_model
from tensorflow.keras.callbacks import EarlyStopping
from scripts.plots import plot_metrics
from IPython.display import display
import gzip
import shutil
import requests
from tqdm import tqdm
import csv


In [2]:
protein_data = read_data()
display(protein_data)

,ID,Accession,Description,Gene,Organism,Sequence
0,001R_FRG3G,Q6GZX4,['DE RecName: Full=Putative transcription fa...,FV3-001R;\n,Frog virus 3 (isolate Goorha) (FV-3).,MAFSAEDVLK EYDRRRRMEA LLLSLYYPND RKLLDYKEWS PP...
1,002L_FRG3G,Q6GZX3,['DE RecName: Full=Uncharacterized protein 0...,FV3-002L;\n,Frog virus 3 (isolate Goorha) (FV-3).,MSIIGATRLQ NDKSDTYSAG PCYAGGCSAF TPRGTCGKDW DL...
2,002R_IIV3,Q197F8,['DE RecName: Full=Uncharacterized protein 0...,IIV3-002R;\n,Invertebrate iridescent virus 3 (IIV-3) (Mosqu...,MASNTVSAQG GSNRPVRDFS NIQDVAQFLL FDPIWNEQPG SI...
3,003L_IIV3,Q197F7,['DE RecName: Full=Uncharacterized protein 0...,IIV3-003L;\n,Invertebrate iridescent virus 3 (IIV-3) (Mosqu...,MYQAINPCPQ SWYGSPQLER EIVCKMSGAP HYPNYYPVHP NA...
4,003R_FRG3G,Q6GZX2,['DE RecName: Full=Uncharacterized protein 3...,FV3-003R;\n,Frog virus 3 (isolate Goorha) (FV-3).,MARPLLGKTS SVRRRLESLS ACSIFFFLRK FCQKMASLVF LN...
...,...,...,...,...,...,...
572209,Z_SABVB,Q6UY62,['DE RecName: Full=RING finger protein Z {EC...,Z {ECO:0000255|HAMAP-Rule:MF_04087};\n,mammarenavirus).,MGNSKSKSKL SANQYEQQTV NSTKQVAILK RQAEPSLYGR HN...
572210,Z_SHEEP,P08105,['DE RecName: Full=Putative uncharacterized ...,NaN,Ovis aries (Sheep).,MSSSLEITSF YSFIWTPHIG PLLFGIGLWF SMFKEPSHFC PC...
572211,Z_TACVF,Q88470,['DE RecName: Full=RING finger protein Z {EC...,Z {ECO:0000255|HAMAP-Rule:MF_04087};\n,Tacaribe virus (strain Franze-Fernandez) (TCRV).,MGNCNRTQKP SSSSNNLEKP PQAAEFRRTA EPSLYGRYNC KC...
572212,Z_TAMVU,A9JR22,['DE RecName: Full=RING finger protein Z {EC...,Z {ECO:0000255|HAMAP-Rule:MF_04087};\n,Tamiami mammarenavirus (isolate Rat/United Sta...,MGLRYSKEVR DRHGDKDPEG RIPITQTMPQ TLYGRYNCKS CW...


In [3]:
# Preprocess the dataset
processed_data = preprocess_data(protein_data)

# Encode sequences
X, y, tokenizer, label_encoder = encode_sequences(processed_data)


KeyError: "['Function'] not in index"

In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = create_model(input_length=X.shape[1], num_classes=y.shape[1])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping]
)

# Save model and encoders
model.save("results/protein_model.h5")

In [ ]:
# Plot metrics
plot_metrics(history)